In [1]:
pip install requests

Note: you may need to restart the kernel to use updated packages.


In [12]:
#code pour scrapper data from apify
import pandas as pd
import requests

# Définir les paramètres de requête
url = "https://api.apify.com/v2/actor-tasks/UG0wXytjHymGDayPP/runs/last/dataset/items"
querystring = {
    "token": "apify_api_LAQKhDgf5JE80y0N85WCXWf3p8np5g2SO3JR",
    "format": "json"
}

# Envoyer la requête GET
response = requests.request("GET", url, params=querystring)

    # Extraire les données JSON de la réponse
data = response.json()

    # Créer un DataFrame pandas à partir des données
df = pd.DataFrame(data)

    # Enregistrer les données dans un fichier Excel
df.to_excel("CIHReviews.xlsx", index=False)


In [10]:
#code pour filtrer les colonne
import pandas as pd

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "/Users/habibaezzagrani/Desktop/CIHReviews.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Définir la liste des valeurs de recherche
valeurs_recherche = ['title', 'categoryName', 'address', 'city', 'url', 'reviewsCount', 'reviewsDistribution', 'name', 'text', 'textTranslated', 'publishedAtDate', 'likesCount', 'reviewerNumberOfReviews', 'stars']

# Filtrer les colonnes du DataFrame
df_filtre = df[valeurs_recherche]

# Créer une liste pour stocker les données filtrées
donnees_filtrees = []

# Filtrer les données pour chaque valeur de recherche
for valeur in valeurs_recherche:
    filtered_data = df_filtre[df_filtre.apply(lambda row: valeur in row.astype(str), axis=1)]
    donnees_filtrees.append(filtered_data)

# Concaténer les données filtrées en un seul DataFrame
donnees_filtrees = pd.concat(donnees_filtrees)


# Enregistrer les données filtrées dans un nouveau fichier Excel
donnees_filtrees.to_excel('CIHReviewsFiltrés.xlsx', index=False)


In [20]:
#code pour filtrer les caracteres speciaux
import pandas as pd
import re

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "/Users/habibaezzagrani/Desktop/DMProject/CIHReviewsFiltrés.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Prétraitement des commentaires
def preprocess_comment(comment):
    comment = str(comment)  # Convertir en chaîne de caractères si nécessaire
    
    # Supprimer les émojis
    comment = comment.encode('ascii', 'ignore').decode('ascii')
    
    # Convertir en minuscules
    comment = comment.lower()
    
    # Supprimer la ponctuation et les caractères spéciaux
    comment = re.sub(r'[^\w\s]', '', comment)
    
    # Supprimer les espaces supplémentaires
    comment = ' '.join(comment.split())
    
    # Filtrer les commentaires décimaux
    if comment.isdecimal():
        return ''
    
    # Filtrer les commentaires commençant par "CIH"
    if comment.startswith('cih'):
        return ''
    
    return comment

# Appliquer le prétraitement aux commentaires de la colonne "text"
df['text'] = df['text'].apply(preprocess_comment)

# Enregistrer le DataFrame mis à jour dans un nouveau fichier Excel
df.to_excel('CIHReviewsPreprocessed.xlsx', index=False)


In [21]:
#code pour filtrer les lignes
import pandas as pd

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "/Users/habibaezzagrani/Desktop/DMProject/CIHReviewsPreprocessed.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Filtrer les lignes vides (supprimer les lignes avec des commentaires vides)
df_filtre = df.dropna(subset=['text'])

# Enregistrer les données filtrées dans un nouveau fichier Excel
df_filtre.to_excel('CIHReviewsFiltrésFinal.xlsx', index=False)

print("Données filtrées enregistrées avec succès dans le fichier 'CIHReviewsFiltrés.xlsx'.")


Données filtrées enregistrées avec succès dans le fichier 'CIHReviewsFiltrés.xlsx'.


In [6]:
#code pour les stop words 
import pandas as pd
from transformers import BertTokenizer, BertModel

# Chemin d'accès au fichier Excel sur votre bureau
chemin_fichier = "/Users/habibaezzagrani/Desktop/DMProject/CIHReviewsFiltrésFinal.xlsx"  # Remplacez "nom_du_fichier.xlsx" par le nom de votre fichier Excel

# Lire le fichier Excel
df = pd.read_excel(chemin_fichier)

# Charger le tokenizer BERT et télécharger les fichiers du modèle si nécessaire
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenizer les commentaires
def tokenize_comment(comment):
    # Appliquer la tokenization avec WordPiece
    tokens = tokenizer.tokenize(comment)
    return tokens

# Appliquer la tokenization aux commentaires de la colonne "text"
df['text_tokens'] = df['text'].apply(tokenize_comment)

# Enregistrer le DataFrame mis à jour dans un nouveau fichier Excel
df.to_excel('CIHReviewsTokenized.xlsx', index=False)


Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [ ]:
!pip install translate 

In [ ]:
#code pour traduction
from translate import Translator

translator = Translator(to_lang="en")

def translate_to_english(text):
    translation = translator.translate(text)
    return translation

df = pd.read_excel('CIHReviewsTokenized.xlsx')

df['colonne_traduite'] = df['text'].apply(translate_to_english)

In [ ]:
#calcule du score de chaque commentaire
import pandas as pd
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# Charger le fichier Excel
df = pd.read_excel('CIHReviewsTokenized.xlsx')

# Instancier le tokenizer et le modèle
tokenizer = AutoTokenizer.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('nlptown/bert-base-multilingual-uncased-sentiment')

# Fonction pour calculer le sentiment
def calculate_sentiment(text):
    tokens = tokenizer.encode(text, return_tensors='pt')
    result = model(tokens)
    sentiment = int(torch.argmax(result.logits))+1
    return sentiment

# Appliquer la fonction calculate_sentiment à la colonne 'textTranslated'
df['sentiment'] = df['textTranslated'].astype(str).apply(calculate_sentiment)

# Afficher le tableau avec la nouvelle colonne 'sentiment'
print(df)
df.to_excel('CIHReviewsTokenized_Sentiment.xlsx', index=False)


In [3]:
#calcule du totaleScore
import pandas as pd

# Charger le fichier Excel initial
df = pd.read_excel('CIHReviewsTokenized_Sentiment.xlsx')


# Calculer le nombre de lignes dans la colonne 'sentiment' groupé par 'address'
result = df.groupby('address')['sentiment'].count()

# Calculer la somme des valeurs dans la colonne 'sentiment' groupée par 'address'
result_sum = df.groupby('address')['sentiment'].sum()

# Calculer le totalScore en divisant la somme par le nombre de lignes pour chaque groupe,
# en arrondissant à un seul chiffre après la virgule
total_score = round(result_sum / result, 1)

# Ajouter la colonne 'totalScore' au DataFrame initial
df['totalScore'] = df['address'].map(total_score)

# Enregistrer les résultats dans un nouveau fichier Excel

df.to_excel('CIHTotalScore.xlsx', index=False)


In [4]:
df[['totalScore']]

,totalScore
0,3.3
1,3.3
2,3.3
3,3.3
4,3.3
...,...
2907,2.1
2908,2.1
2909,2.1
2910,2.1


In [5]:
import pandas as pd

# Charger le fichier Excel
df = pd.read_excel('/Users/habibaezzagrani/Desktop/DMProject/CIHTotalScore.xlsx')

# Supprimer une colonne spécifique
#col_to_drop = 'text_tokens'
#df = df.drop(col_to_drop, axis=1)
col_to_drop = 'text'
df = df.drop(col_to_drop, axis=1)
col_to_drop = 'name'
df = df.drop(col_to_drop, axis=1)
col_to_drop = 'stars'
df = df.drop(col_to_drop, axis=1)
col_to_drop = 'likesCount'
df = df.drop(col_to_drop, axis=1)
col_to_drop = 'reviewerNumberOfReviews'
df = df.drop(col_to_drop, axis=1)
# Enregistrer le DataFrame dans un nouveau fichier Excel
df.to_excel('/Users/habibaezzagrani/Desktop/DMProject/NVCIHTotalScore.xlsx', index=False)

In [7]:
#code pour merging
import pandas as pd

# Charger le fichier Excel
df = pd.read_excel('/Users/habibaezzagrani/Desktop/DMProject/NVCIHTotalScore.xlsx')

# Sélectionner les colonnes à regrouper
cols_to_group = ['colonne_traduite', 'publishedAtDate','sentiment']

# Regrouper les colonnes par adresse (address) et les rassembler dans des listes
grouped_df = df.groupby('address')[cols_to_group].apply(
    lambda x: [[ tokens, date,score] for tokens, date,score in zip(
         x['colonne_traduite'].tolist(), x['publishedAtDate'].tolist(),x['sentiment'].tolist())]
).reset_index()

# Fusionner les données regroupées avec le DataFrame d'origine
merged_df = pd.merge(df, grouped_df, on='address', how='left')

# Supprimer les colonnes d'origine
merged_df = merged_df.drop(cols_to_group, axis=1)

# Supprimer les lignes en double pour la colonne 'address'
merged_df = merged_df.drop_duplicates(subset=['address'])

# Enregistrer le DataFrame fusionné dans un nouveau fichier Excel
merged_df.to_excel('/Users/habibaezzagrani/Desktop/DMProject/FinalFile.xlsx', index=False)


In [5]:
!pip install --upgrade torch torchvision

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 MB 1.3 MB/s eta 0:00:0000:0100:02
  Using cached torchvision-0.15.2-cp310-cp310-macosx_10_9_x86_64.whl (1.5 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.12.1
    Uninstalling torch-1.12.1:
      Successfully uninstalled torch-1.12.1


In [1]:
!pip install --upgrade transformers

In [1]:
!pip install --upgrade scipy numpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.1/35.1 MB 838.4 kB/s eta 0:00:0000:0100:02
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.8/19.8 MB 1.5 MB/s eta 0:00:0000:0100:01m
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.0
    Uninstalling scipy-1.10.0:
      Successfully uninstalled scipy-1.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gensim 4.3.0 requires FuzzyTM>=0.4.0, which is not installed.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.24.3 which is incompatible.


In [7]:
import pandas as pd
import psycopg2

# Load data from Excel file
excel_file_path = "/Users/habibaezzagrani/Desktop/DMProject/CIHTotalScore.xlsx"
sheet_name = "city"  # Specify the sheet name if applicable
columns_to_load = ['city']  # Specify the column names to load

# Read specific columns from Excel into a pandas DataFrame
df = pd.read_excel(excel_file_path, sheet_name=sheet_name, usecols=columns_to_load)

# Database connection details
db_host = "localhost"
db_name = "cihdatawarehouse"
db_user = "habibaezzagrani"
db_password = ""
db_port = "5432"

# Establish a connection to the PostgreSQL database
conn = psycopg2.connect(
    host=db_host,
    database=db_name,
    user=db_user,
    password=db_password,
    port=db_port
)

# Create a cursor object
cur = conn.cursor()

# Create the table in PostgreSQL
table_name = "CityDimension"
cur.execute(f"CREATE TABLE IF NOT EXISTS {table_name} (id SERIAL PRIMARY KEY,city TEXT)")

# Convert DataFrame columns to a list of tuples
records = df.to_records(index=False)
values = list(records)

# Create the insert query
insert_query = f"INSERT INTO {table_name} (column1) VALUES (%s)"

# Execute the insert query with the DataFrame values
cur.executemany(insert_query, values)

# Commit the changes to the database
conn.commit()

# Close the cursor and database connection
cur.close()
conn.close()

print("Data loaded successfully to PostgreSQL!")


ModuleNotFoundError: No module named 'pandas'